In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point

In [ ]:
latlong = pd.read_csv('latlong.csv')

In [ ]:
latlong2 = pd.read_csv('latlong2.csv')

In [ ]:
geometry = [Point(xy) for xy in zip(latlong.Longitude, latlong.Latitude)]

In [ ]:
geo_df = gpd.GeoDataFrame(geometry = geometry)

In [ ]:
geo_df['geometry'] = geo_df['geometry'][geo_df['geometry'].x < -110]

In [ ]:
latlong['coords'] = list(zip(latlong.Longitude,latlong.Latitude))

In [ ]:
latlong = latlong.drop(['Latitude','Longitude'],axis=1,)

In [ ]:
latlong.dropna().to_csv('stuff_fixed.csv')

In [ ]:
latlong['coords'] = latlong['coords'][latlong['coords'] < (-110,1000)]

In [ ]:
import geopandas as gpd
from pyproj import Transformer
import matplotlib.pyplot as plt

geojson_path = 'California_Zip_Codes.geojson'

gdf = gpd.read_file(geojson_path)

gdf = gdf.set_geometry('geometry')

transformer = Transformer.from_crs(gdf.crs, "EPSG:4326", always_xy=True)

gdf = gdf.to_crs("EPSG:4326")

gdf["latitude"] = gdf["geometry"].apply(lambda geom: geom.centroid.y)
gdf["longitude"] = gdf["geometry"].apply(lambda geom: geom.centroid.x)

#fig, ax = plt.subplots(figsize=(10, 10))
#gdf.plot(ax=ax, edgecolor='black', linewidth=0.5)
#plt.title("GeoJSON File with Lat/Lon Coordinates")
#plt.xlabel("Longitude")
##plt.ylabel("Latitude")
#plt.show()

In [ ]:
geo_df.plot(ax=gdf.plot(),marker='o', color='red', markersize=1) #plot grocery stores

In [ ]:
zipcodes = list(latlong2['Zip Codes'])

In [ ]:
weights = []
radii = []

In [ ]:
good = pd.read_csv('good2.csv')
good = good.drop(['Transportation data normalized','Ratio of Incomes','Pop per sq mile, filtered for neg and aligned for zip codes','Weight?','max radius'],axis=1)
good = good.drop_duplicates('ZIP_CODE')
good = good.set_index('ZIP_CODE')

In [ ]:
for zipcode in zipcodes:
    weights.append(good.loc[zipcode].weight)
    radii.append(good.loc[zipcode].radius)

In [ ]:
weights_df = pd.DataFrame(columns=['weight'])

In [ ]:
weights_df['weight'] = weights #replace with list of weights

In [ ]:
dist = radii #replace with list of radii in miles

In [ ]:
dist2 = [x/69 for x in dist] #converts miles to degrees for buffer

In [ ]:
new_df = geo_df.copy()
for i in range(len(geo_df['geometry'])):
    try:
        new_df['geometry'][i] = new_df['geometry'][i].buffer(dist2[i])
    except AttributeError:
        continue
new_df['weight'] = weights

In [ ]:
new_df.plot(ax=gdf.plot(),marker='o', cmap='Reds', markersize=10,column='weight',alpha=1) #with weight

In [ ]:
new_df.plot(ax=gdf.plot(),marker='o', markersize=0,color='Red') #without weight